# Setup

In [46]:
import tensorflow as tf
import numpy as np
import tf_agents
import matplotlib.pyplot as plt

from tf_agents.environments import tf_py_environment

from tf_env.UR_ENV import UR_env

# ff

In [47]:
train_env_py= UR_env()
eval_env_py=UR_env()

train_env_tf=tf_py_environment.TFPyEnvironment(train_env_py)
eval_env_tf=tf_py_environment.TFPyEnvironment(eval_env_py)

In [48]:
from tf_agents.train.utils import strategy_utils
use_gpu = True
strategy = strategy_utils.get_strategy(tpu=False, use_gpu=use_gpu)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [49]:
from tf_agents.train.utils import spec_utils
from tf_agents.agents.ddpg import critic_network

critic_joint_fc_layer_params = (256, 256)

observation_spec, action_spec, time_step_spec = (
      spec_utils.get_tensor_specs(train_env_tf))

with strategy.scope():
  critic_net = critic_network.CriticNetwork(
        (observation_spec, action_spec),
        observation_fc_layer_params=None,
        action_fc_layer_params=None,
        joint_fc_layer_params=critic_joint_fc_layer_params,
        kernel_initializer='glorot_uniform',
        last_kernel_initializer='glorot_uniform')

In [50]:
from tf_agents.networks import actor_distribution_network
from tf_agents.agents.sac import tanh_normal_projection_network

actor_fc_layer_params = (256, 256)

with strategy.scope():
  actor_net = actor_distribution_network.ActorDistributionNetwork(
      observation_spec,
      action_spec,
      fc_layer_params=actor_fc_layer_params,
      continuous_projection_net=(
          tanh_normal_projection_network.TanhNormalProjectionNetwork))

In [51]:
from tf_agents.train.utils import train_utils
from tf_agents.agents.sac import sac_agent

critic_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
alpha_learning_rate = 3e-4 # @param {type:"number"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

with strategy.scope():
  train_step = train_utils.create_train_step()

  tf_agent = sac_agent.SacAgent(
        time_step_spec,
        action_spec,
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.keras.optimizers.Adam(
            learning_rate=actor_learning_rate),
        critic_optimizer=tf.keras.optimizers.Adam(
            learning_rate=critic_learning_rate),
        alpha_optimizer=tf.keras.optimizers.Adam(
            learning_rate=alpha_learning_rate),
        target_update_tau=target_update_tau,
        target_update_period=target_update_period,
        td_errors_loss_fn=tf.math.squared_difference,
        gamma=gamma,
        reward_scale_factor=reward_scale_factor,
        train_step_counter=train_step)

  tf_agent.initialize()

In [52]:
from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer_capacity = 100 # @param {type:"integer"}
batch_size = 50 # @param {type:"integer"}

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    tf_agent.collect_data_spec,
    batch_size=batch_size,
    max_length=replay_buffer_capacity)

replay_observer = [replay_buffer.add_batch]

In [53]:
from tf_agents.policies import py_tf_eager_policy

tf_eval_policy = tf_agent.policy
eval_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_eval_policy, use_tf_function=True)

In [54]:
tf_collect_policy = tf_agent.collect_policy
collect_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_collect_policy, use_tf_function=True)

In [55]:
from tf_agents.policies import random_py_policy

random_policy = random_py_policy.RandomPyPolicy(
  train_env_py.time_step_spec(), train_env_py.action_spec())

In [60]:
from tf_agents.train import actor

initial_collect_steps = 100 # @param {type:"integer"}

bf = []

br=[bf.append]

initial_collect_actor = actor.Actor(
  train_env_py,
  random_policy,
  train_step,
  steps_per_run=initial_collect_steps,
  observers=br)
initial_collect_actor.run()

InvalidArgumentError: {{function_node __wrapped__ResourceScatterUpdate_device_/job:localhost/replica:0/task:0/device:CPU:0}} Must have updates.shape = indices.shape + params.shape[1:] or updates.shape = [], got updates.shape [2,3], indices.shape [50], params.shape [5000,2,3] [Op:ResourceScatterUpdate]